In [ ]:
# dF/F analysis, Save file as CSV, Plotting the data

import pandas as pd
import numpy as np
import plotly.graph_objects as go
import os

# Define the file path as a string
filename = "E:/199-200_10-17-18Oct-2024-DT-LT_Circular-Arena/Analysis/Rectangular Arena/Ca-Signal/200-trial2-opto-raw-signal.csv"

# Ensure the file exists and handle errors
try:
    data = pd.read_csv(filename)
except FileNotFoundError:
    print(f"Error: File {filename} not found.")
    exit()
except pd.errors.EmptyDataError:
    print(f"Error: File {filename} is empty.")
    exit()
except pd.errors.ParserError:
    print(f"Error: File {filename} is not in a valid CSV format.")
    exit()

# Display the first few rows to verify the data
print(data.head())

# Identify cell columns (assuming they are named 'Cell-07', 'Cell-12', etc.)
cell_columns = [col for col in data.columns if col.startswith('cell-')]

# Check if cell columns are identified
print("Cell columns identified:", cell_columns)

# Extract time and fluorescence data
time = data.iloc[:, 0]  # First column assumed to be time
F = data[cell_columns]  # Fluorescence traces for each cell

# Extracting cell traces as a list for easier processing
F_cells = [F.iloc[:, i] for i in range(F.shape[1])]

# Preallocate lists and a dictionary for results
DF_all = []
DF_dict = {}  # Dictionary to hold DF/F data for each cell

# Function to plot the original signal
def plot_original_signal(time, original_signal, cell_name):
    """
    Plots the original fluorescence signal for a given cell.

    Parameters:
    time (pd.Series): Time points for the signal.
    original_signal (pd.Series): Original fluorescence signal.
    cell_name (str): Identifier for the cell being plotted.
    """
    fig = go.Figure()

    # Add original fluorescence trace
    fig.add_trace(go.Scatter(
        x=time,
        y=original_signal,
        mode='lines',
        name=f'Original Signal ({cell_name})',
        line=dict(color='blue', width=1)
    ))

    # Update layout for better visualization
    fig.update_layout(
        title=f"Original Fluorescence Signal for {cell_name}",
        xaxis_title='Time (s)',
        yaxis_title='Fluorescence Intensity',
        legend_title='Signal Type'
    )

    # Display the figure
    fig.show()

# Function to plot the dF/F signal
def plot_dF_F_signal(time, dF_F, cell_name):
    """
    Plots the dF/F signal for a given cell.

    Parameters:
    time (pd.Series): Time points for the signal.
    dF_F (pd.Series): Calculated dF/F signal.
    cell_name (str): Identifier for the cell being plotted.
    """
    fig = go.Figure()

    # Add dF/F trace
    fig.add_trace(go.Scatter(
        x=time,
        y=dF_F,
        mode='lines',
        name=f'dF/F ({cell_name})',
        line=dict(color='red', width=1)
    ))

    # Update layout for better visualization
    fig.update_layout(
        title=f"dF/F Signal for {cell_name}",
        xaxis_title='Time (s)',
        yaxis_title='dF/F',
        legend_title='Signal Type'
    )

    # Display the figure
    fig.show()

# Loop through each cell trace
for ii, F_cell in enumerate(F_cells):
    if len(F_cell) < 2:
        print(f"Skipping cell {cell_columns[ii]} due to insufficient data points.")
        continue

    # Avoid division by zero in normalization step
    min_val = F_cell.min()
    if min_val == 0:
        print(f"Warning: Min value is zero for cell {cell_columns[ii]}, skipping normalization.")
        continue

    # Calculate delta F/F (normalized fluorescence)
    DF = (F_cell - min_val) / abs(min_val)

    # Plot the original signal for the cell
    plot_original_signal(time, F_cell, cell_columns[ii])

    # Plot the dF/F signal for the cell
    plot_dF_F_signal(time, DF, cell_columns[ii])

    # Append the normalized data for further processing
    DF_all.append(DF)

    # Save the DF/F values in the dictionary with cell name as key
    DF_dict[cell_columns[ii]] = DF

# Create a DataFrame from the time and DF_dict
df_final = pd.DataFrame(DF_dict)
df_final.insert(0, 'time', time)  # Insert time as the first column

# Define the output file path
output_dir = "E:/199-200_10-17-18Oct-2024-DT-LT_Circular-Arena/Analysis/Rectangular Arena/Ca-Signal/"
output_filename = "200-trial2-opto-delta_f_over_f_cells.csv"
output_filepath = os.path.join(output_dir, output_filename)

# Save the DataFrame to CSV
df_final.to_csv(output_filepath, index=False)

# Output the path of the saved file
print(f"Delta F/F data saved to {output_filepath}")
